In [1]:
import numpy as np
import os
import sys
import pandas as pd

import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

import pylab as pl
import h5py

import tensorflow as tf
from tensorflow.keras import mixed_precision

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
os.environ['CUDA_VISIBLE_DEVICES']="0" 

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


from sklearn.metrics import confusion_matrix


base_dir = '/media/tord/T7/Thesis_ssd/MasterThesis3'
os.chdir(base_dir)
from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.HelperFunctions import HelperFunctions
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.TimeAugmentor import TimeAugmentor
from Classes.DataProcessing.NoiseAugmentor import NoiseAugmentor
from Classes.DataProcessing.RamLoader import RamLoader
from Classes.DataProcessing.RamGenerator import RamGenerator
from Classes.Modeling.InceptionTimeModel import InceptionTimeModel
from Classes.Modeling.InceptionTimeRGS import InceptionTimeRGS
from Classes.Modeling.CustomCallback import CustomCallback
from Classes.Modeling.ResultFitter import ResultFitter
from Classes.Scaling.ScalerFitter import ScalerFitter
from Classes.Scaling.MinMaxScalerFitter import MinMaxScalerFitter
from Classes.Scaling.StandardScalerFitter import StandardScalerFitter
import json
#from Classes import Tf_shutup
#Tf_shutup.Tf_shutupe()


plt.rcParams["figure.figsize"]= (15,15)
helper = HelperFunctions()

import sys
ISCOLAB = 'google.colab' in sys.modules

import random
import pprint

# Equivalent to the two lines above
mixed_precision.set_global_policy('mixed_float16')



%load_ext tensorboard

1 Physical GPUs, 1 Logical GPUs
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 3090, compute capability 8.6


In [2]:
load_args = {
    'earth_explo_only' : True,
    'noise_earth_only' : False,
    'noise_not_noise' : False,
    'downsample' : True,
    'upsample' : True,
    'frac_diff' : 1,
    'seed' : 1,
    'subsample_size' : 0.4,
    'balance_non_train_set' : True,
    'use_true_test_set' : False
}
loadData = LoadData(**load_args)
full_ds, train_ds, val_ds, test_ds = loadData.get_datasets()
noise_ds = loadData.noise_ds
handler = DataHandler(loadData)

if load_args['earth_explo_only']:
    full_and_noise_ds = np.concatenate((full_ds, noise_ds))
    timeAug = TimeAugmentor(handler, full_and_noise_ds, seed = load_args['seed'])
else:
    timeAug = TimeAugmentor(handler, full_ds, seed = load_args['seed'])

In [3]:
noise_ds

array([[ '/media/tord/T7/Thesis_ssd/norsar_data_nov/noise/2020-05-30T21.47.41.000000Z.h5',
        'noise', 0],
       [ '/media/tord/T7/Thesis_ssd/norsar_data_nov/noise/2001-05-13T02.24.59.000000Z.h5',
        'noise', 0],
       [ '/media/tord/T7/Thesis_ssd/norsar_data_nov/noise/2020-02-21T02.49.34.000000Z.h5',
        'noise', 0],
       ..., 
       [ '/media/tord/T7/Thesis_ssd/norsar_data_nov/noise/2010-07-22T00.23.43.000000Z.h5',
        'noise', 0],
       [ '/media/tord/T7/Thesis_ssd/norsar_data_nov/noise/2012-05-12T11.04.23.000000Z.h5',
        'noise', 0],
       [ '/media/tord/T7/Thesis_ssd/norsar_data_nov/noise/2012-05-10T02.53.24.000000Z.h5',
        'noise', 0]], dtype=object)

In [4]:
print(len(train_ds), len(val_ds), len(test_ds))
classes, counts = handler.get_class_distribution_from_ds(full_ds)
classes, counts = handler.get_class_distribution_from_ds(train_ds)
classes, counts = handler.get_class_distribution_from_ds(val_ds)
print("Nr noise samples " + str(len(loadData.noise_ds)))

62648 12529 8354
Total: 83531, earthquake: 42458, explosion: 41073
Total: 62648, earthquake: 31903, explosion: 30745
Total: 12529, earthquake: 6380, explosion: 6149
Nr noise samples 42391


In [5]:
hyper_grid = {    
    "batch_size" : [128, 256, 512, 1024],
    "epochs" : [100, 100],
    "learning_rate" : [0.1, 0.01, 0.001, 0.0001],
    "optimizer" : ["adam", "rmsprop", "sgd"]
    }
model_grid = {
    "use_residuals" : [True, False],
    "use_bottleneck" : [True, False],
    "nr_modules" : [1, 3, 6, 9, 12],
    "kernel_size" : [20, 30, 40, 50],
    "bottleneck_size" : [ 26, 28, 30, 32, 34, 36],
    "num_filters" : [24, 26, 28, 30, 32, 34, 36, 38, 40 ,42],
    "shortcut_activation" : ["relu", "sigmoid", "softmax", "tanh"],
    "module_activation" : ["linear", "relu", "sigmoid", "softmax", "tanh"],
    "module_output_activation" : ["relu", "linear", "sigmoid", "softmax", "tanh"],
    "output_activation": ["sigmoid"],
    "reg_shortcut": [True, False],
    "reg_module" : [True, False],
    "l1_r" : [0.1, 0.001, 0.0001, 0],
    "l2_r" : [0.1, 0.001, 0.0001, 0]
}

num_channels = 3

use_time_augmentor = True
use_scaler = True
use_noise_augmentor = True
detrend = False
use_minmax = False
use_highpass = True
highpass_freq = 0.1

n_picks = 50

use_tensorboard = False
use_liveplots = False
use_custom_callback = False
use_early_stopping = True
start_from_scratch = False
use_reduced_lr = True

randomGridSearch = InceptionTimeRGS(loadData, train_ds, val_ds, detrend, use_scaler, use_time_augmentor, 
                                    use_noise_augmentor, use_minmax, use_highpass, n_picks, hyper_grid = hyper_grid, 
                                    model_grid = model_grid, use_tensorboard = use_tensorboard,
                                    use_liveplots = use_liveplots, use_custom_callback = use_custom_callback,
                                    use_early_stopping = use_early_stopping, highpass_freq = highpass_freq,
                                    start_from_scratch = start_from_scratch, use_reduced_lr = use_reduced_lr, 
                                    num_channels = num_channels)

Note that highpass and detrend have not been implemented yet in this class.


In [6]:
def clear_tensorboard_dir():
    import os
    import shutil
    path = f"{base_dir}/Tensorboard_dir/fit"
    files = os.listdir(path)
    print(files)
    for f in files:
        shutil.rmtree(os.path.join(path,f))
if use_tensorboard:
    clear_tensorboard_dir()
    %tensorboard --logdir tensorboard_dir/fit

In [7]:
results_df, min_loss, max_accuracy, max_precision, max_recall = randomGridSearch.fit()

Length of grid: 96
Length of grid: 30720000
Fit process completed after 46.224916219711304 seconds. Total datapoints fitted: 125922.
Average time per datapoint: 0.0003670916616612769
Finished fitting augmentors and scaler
Starting loading to RAM
Completed loading to RAM
Starting loading to RAM
Completed loading to RAM


NameError: name 'mixed_precision' is not defined